In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import seq2seq

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(seq2seq)    
net = seq2seq.Seq2SeqRNN(char2i, i2char)
net.to(device)
net.fit(source_1, target_1, save_path = "checkpoints/seq2seq_rnn.pt")

Model: Seq2Seq RNN
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 8
Out embedding dimension: 8
Encoder hidden units: 32
Encoder layers: 2
Decoder hidden units: 32
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 40,495

Epoch | Train                 | Training time
      | Loss     | Error Rate |
---------------------------------------------
    1 |   3.0202 |     70.035 |         37.28
    2 |   2.1847 |     55.512 |         74.30
    3 |   1.8134 |     48.153 |        114.33
    4 |   1.6475 |     44.843 |        154.92
    5 |   1.5552 |     43.085 |        193.19


,epoch,train_loss,train_error_rate,training_time,learning_rate,weight_decay,model,in_embedding_dimension,out_embedding_dimension,encoder_hidden_units,encoder_layers,decoder_hidden_units,decoder_layers,dropout,parameters
0,1,3.020154,70.035337,37.283324,0.0001,0,Seq2Seq RNN,8,8,32,2,32,2,0.0,40495
1,2,2.184674,55.511870,74.295685,0.0001,0,Seq2Seq RNN,8,8,32,2,32,2,0.0,40495
2,3,1.813378,48.153104,114.333292,0.0001,0,Seq2Seq RNN,8,8,32,2,32,2,0.0,40495
3,4,1.647490,44.843041,154.919591,0.0001,0,Seq2Seq RNN,8,8,32,2,32,2,0.0,40495
4,5,1.555250,43.084522,193.194491,0.0001,0,Seq2Seq RNN,8,8,32,2,32,2,0.0,40495


In [9]:
importlib.reload(seq2seq)    
net = seq2seq.Seq2SeqRNN(char2i, i2char)
net.load_state_dict(torch.load("checkpoints/seq2seq_rnn.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(source_1[:5])

[net.tensor2text(t) for t in indexes]

Model: Seq2Seq RNN
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 8
Out embedding dimension: 8
Encoder hidden units: 32
Encoder layers: 2
Decoder hidden units: 32
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 40,495



[['<START> the the the the the',
  '<START> the the the the to ',
  '<START>s the the the the th',
  '<START> the the the the Kt-',
  '<START> the the the the thi'],
 ['<START> the the the the the',
  '<START>e the the the the th',
  '<START>e the the the the to',
  '<START>e the the the the pa',
  '<START>e the the the the Kt'],
 ['<START>ing the the the the ',
  '<START>hite the the the the',
  '<START>he the the the the t',
  '<START>er the the the the t',
  '<START>hite the the the to '],
 ['<START>the the the the the ',
  '<START>the the the the and ',
  '<START>the the Black the th',
  '<START>the the the the Kt-K',
  '<START>the the the the Kt-Q'],
 ['<START>. Kt-K4, the the the',
  '<START>. Kt-B3, the the the',
  '<START>. Kt-K4, the the Kt-',
  '<START>. Kt-B3, the the Kt-',
  '<START>. Kt-K4, the the to ']]

In [10]:
importlib.reload(seq2seq)    
net = seq2seq.Seq2SeqRNN(char2i, i2char)
net.load_state_dict(torch.load("checkpoints/seq2seq_rnn.pt"))
net.to(device)
indexes, log_probabilities = net.sample(source_1[:5])

net.tensor2text(indexes)

Model: Seq2Seq RNN
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 8
Out embedding dimension: 8
Encoder hidden units: 32
Encoder layers: 2
Decoder hidden units: 32
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 40,495



['<START>n sins as uvere colf',
 '<START> mowecey at in Q. Bx',
 '<START>is cows\nQiSt iy the ',
 '<START>harotg hp.\n\n        ',
 '<START>. B-K2\n, Kt-K1; BxP,']